# Dependences

In [1]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes>=0.39.0
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 47.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 8.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uaf34nd3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uaf34nd3
  Resolved https://github.com/huggingface/transformers.git to commit 30ed3adf474aaf2972ab56f5624089bc24a6adf3
  Installing bu

In [1]:
!nvidia-smi

Thu Jul 27 15:28:31 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070         Off| 00000000:01:00.0 Off |                  N/A |
| 44%   57C    P8               16W / 220W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!free

               total        used        free      shared  buff/cache   available
Mem:        65761324     1063672    47848760        4072    16848892    63994368
Swap:        2097148           0     2097148


# Imports

In [1]:
import os
import torch
import time
import datetime
import sys
import traceback
import gc

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

# Loading the model

In [2]:
model_name = 'legendhasit/xgen-7b-8k-inst-8bit'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/linagora/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/linagora/anaconda3/lib/libcudart.so'), PosixPath('/home/linagora/anaconda3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install huggingface_hub["cli"]
!huggingface-cli delete-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.5 MB/s eta 0:00:00
? Select revisions to delete: 0 revisions selected counting for 0.0. 
❯ ○ None of the following (if selected, nothing will be deleted).
 
Model bert-base-multilingual-cased (715.3M, used 3 days ago)
  ○ fdfce55e: main # modified 1 week ago
 
Model OpenLLM-France/bloom-560m-ecommerce-faq-fr (12.6M, used 3 days ago)
  ○ 239462ee: (detached) # modified 4 weeks ago
  ○ 3b13cb96: main # modified 4 weeks ago
 
Model Salesforce/xgen-7b-8k-inst (44.4K, used 3 days ago)
  ○ 5fe0f1b2: (detached) # modified 3 weeks ago
  ○ 68f77dec: main # modified 6 days ago
 
Model bigscience/bloom-560m (1.1G, used 3 days ago)
  ○ 4f42c91d: main # modified 4 weeks ago
 
Model legendhasit/xgen-7b-8k-inst-8bit (13.8G, used 55 minutes ago)
  ○ 0fced741: main # modified 3 weeks ago

 
i49mPress <space> to select, <enter> to validate and <ctrl+c> to quit without modif
? Select revisions to delete: 0 revisions selected counting for 0.0. 


# Inference

In [3]:
# Parameters:
# - DATASET_NAME
# - HEADER
# - PROMPT_TEMPLATE
# - samples in dataset 'input/<dataset_name>'
# - instructions in 'instructions.txt'

def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

mkdir('input')

# Define prompt template
# ==========================================================================================
HEADER = (
    "A chat between a curious human and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the human's questions.\n\n"
)

EXAMPLE_TEMPLATE = """### Human: {instruction}

{text}

### Assistant: {answer}"""

REQUEST_TEMPLATE = """### Human: {instruction}

{text}

### Assistant:"""

# Load texts from input folder
# ==========================================================================================
texts_list = os.listdir('input/texts')
texts = [text for text in texts_list]
texts.sort()
n_texts = len(texts)
print('-- Found', n_texts, 'texts:', texts)

# Load instructions
# ==========================================================================================
mkdir('instructions')
type_folder = os.listdir('input/instructions')
print("Types of prompts: ")
instruction_types = []
n_instructions = 0
for subfolder in type_folder:
    instruction_file = open('input/instructions/'+subfolder+'/pro_'+subfolder+'.txt', 'r', encoding='utf-8')
    prompts = instruction_file.readlines()
    n_prompts = len(prompts)
    n_instructions += n_prompts
    for i in range(n_prompts):
        prompts[i] = prompts[i].replace('\n', '')
    instruction_types.append(prompts)
    instruction_file.close()
    print(prompts)
    print()

-- Found 1 texts: ['txt_ami.txt']
Types of prompts: 
['Summarize the following text.', 'Summarize.', 'Provide a structured summary with 5 paragraphs of the following text.', 'Summarize the following text writing 5 paragraphs.', 'Summarize the following text writing 5 paragraphs which contain all the important details']

['Summarize the following text.', 'Summarize.', 'Give the key points in the following text.', 'Provide a list of key points for this text, containing all the important details.', 'As a secretary, write down the minutes of the following meeting.']

['Summarize the following text.', 'Summarize.', 'Provide a one paragraph summary of the following text.', 'Summarize the following text as one paragraph.', 'Provide a one paragraph summary of the following text, containing all the important details.']



In [7]:
# Inference
# ==========================================================================================
initial_time = time.time()
skipped_samples = 0

n_samples = 4 # A CHANGER A 4
n_inferences_total = n_instructions * n_texts * n_samples # Total number of inferences
n_inference = 0

def TwoShotHeader(k, instruction): # k désigne le type de prompt (1, 2 ou 3). Renvoie un header avec deux exemples.
    file_example = open('input/examples/txt_gpt.txt', 'r', encoding='utf-8')
    text_example = file_example.read()
    file_example.close()
    file_sum = open('input/summaries/'+str(k)+'/sum_'+str(k)+'_gpt.txt', 'r', encoding='utf-8')
    text_sum = file_sum.read()
    file_sum.close()
    example_1 = EXAMPLE_TEMPLATE.format(instruction=instruction, text=text_example, answer=text_sum)
    file_example = open('input/examples/txt_fre.txt', 'r', encoding='utf-8')
    text_example = file_example.read()
    file_example.close()
    file_sum = open('input/summaries/'+str(k)+'/sum_'+str(k)+'_fre.txt', 'r', encoding='utf-8')
    text_sum = file_sum.read()
    file_sum.close()
    example_2 = EXAMPLE_TEMPLATE.format(instruction=instruction, text=text_example, answer=text_sum)

    return HEADER + example_1 + "\n\n" + example_2 + "\n\n" 

def treatOutput(output, i, n_inference, sample): # DOIT PRENDRE EN ENTREE L'OUTPUT DETOKENIZED ET LE TOKENIZED (sample)
    mkdir('output')
    mkdir('output/'+str(i))
    target_path = 'output/'+str(i)+'/sum_'+str(i)+'_'+str(n_inference)+'.txt' # Format sum_2_9.txt
    text_file = open(target_path, 'w', encoding='utf-8')
    occ_1 = output.find("### Assistant: ")
    occ_2 = output.find("### Assistant: ", occ_1+1)
    occ_3 = output.find("### Assistant: ", occ_2+1)
    output = output[occ_3+15:]
    if output.find('<|endoftext|>')!=-1:
        output = output[:-14]
    text_file.write(str(len(sample[0]))+'\n'+output)
    text_file.close()

mkdir('intermediate')
print('Starting computation...')
for i in range(len(instruction_types)):
    instructions = instruction_types[i]
    mkdir('intermediate/'+str(i+1))
    for name_text in texts:
        # Pour un prompt donné, un texte donné:
        text_file = open('input/texts/'+name_text, 'r', encoding='utf-8')
        text_request = text_file.read()
        text_file.close()
        # Construction des exemples
        for instruction in instructions:
            two_shot_header = TwoShotHeader(i+1, instruction)
            user_request = REQUEST_TEMPLATE.format(instruction=instruction, text=text_request)
            prompt = two_shot_header + user_request
            for j in range(n_samples): # for j in range(n_samples): # puis tout décaler
                n_inference += 1
                ### DISPLAY :
                print("Inférence "+str(n_inference)+"/"+str(n_inferences_total)+":")
                if n_inference>1: # Pour une estimation, il faut forcément attendre le 2ème passage
                    approx_total = n_inferences_total * (time.time() - initial_time) / (n_inference-1)
                    approx_remaining = approx_total * (1 - (n_inference-1)/n_inferences_total)
                    print('-- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')

                target_path = 'intermediate/'+str(i+1)
                mkdir(target_path)
                target_path += '/sum_'+str(i+1)+'_'+str(n_inference)+'.txt' # Format sum_2_9.txt
                if os.path.isfile(target_path):
                    print('-- Found intermediate result file \'' + target_path + '\', skipped.')
                    skipped_samples += 1
                    continue

                try:
                    input_ids = tokenizer(prompt, return_tensors="pt").to('cuda') # ICI RESTRICTION sur prompt
                    input_length = len(input_ids['input_ids'][0])
                    print('-- Input Length:', input_length)
                    sample = model.generate(**input_ids, do_sample=True, max_new_tokens=700, top_k=20, eos_token_id=50256, temperature=0.3) # Top-k, température, max new tokens
                    output = tokenizer.decode(sample[0]).strip()
                    treatOutput(output, i+1, n_inference, sample)
                    # Save answer in file
                    target_file = open(target_path, 'w', encoding='utf-8')
                    target_file.write(output)
                    target_file.close()

                    del input_ids
                    del sample
                    del output
                except:
                    print('Could not compute '+str(i)+'_'+str(j)+' prompt.')
                    #print(prompt) (trop long comme prompt)
                    traceback.print_exc()

                gc.collect()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

Starting computation...
Inférence 1/60:
-- Found intermediate result file 'intermediate/1/sum_1_1.txt', skipped.
Inférence 2/60:
-- Estimated Remaining Time: 0:00:00 (total 0:00:00)
-- Found intermediate result file 'intermediate/1/sum_1_2.txt', skipped.
Inférence 3/60:
-- Estimated Remaining Time: 0:00:00 (total 0:00:00)
-- Found intermediate result file 'intermediate/1/sum_1_3.txt', skipped.
Inférence 4/60:
-- Estimated Remaining Time: 0:00:00 (total 0:00:00)
-- Found intermediate result file 'intermediate/1/sum_1_4.txt', skipped.
Inférence 5/60:
-- Estimated Remaining Time: 0:00:00 (total 0:00:00)
-- Found intermediate result file 'intermediate/1/sum_1_5.txt', skipped.
Inférence 6/60:
-- Estimated Remaining Time: 0:00:00 (total 0:00:00)
-- Input Length: 16
Inférence 7/60:
-- Estimated Remaining Time: 0:00:15 (total 0:00:17)
-- Input Length: 16
Inférence 8/60:
-- Estimated Remaining Time: 0:00:26 (total 0:00:30)
-- Input Length: 16
Inférence 9/60:
-- Estimated Remaining Time: 0:00:34

# Calculs de scores

In [2]:
!pip install rouge_score rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu
!pip install nltk

  Preparing metadata (setup.py) ... done
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=333555242f4e464ad2f1466233462741b2f13d4d85af3a6bfe866ca7f1f269f6
  Stored in directory: /home/linagora/.cache/pip/wheels/3e/94/5c/7ff8a51c53c1bbc8df4cac58aa4990ffbc6fa203e9f0808fdd
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 29.0 MB/s eta 0:00:00
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached pyarrow-12.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 12.6 MB/s eta 0:00:00


In [3]:
import evaluate
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/linagora/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/linagora/.local/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/linagora/anaconda3/lib/libcudart.so.11.0'), PosixPath('/home/linagora/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [8]:
leprompt = """In this conversation, Valérie Pécresse, Éric Zemmour, and the moderator discuss various economic proposals and reforms. The main topic of debate is Emmanuel Macron's proposal to raise the legal retirement age to sixty-five. Pécresse accuses Macron of previously mismanaging funds but now trying to replenish the pension system. Zemmour mocks the proposal, suggesting an even higher retirement age. 

The conversation moves on to other economic issues, with Pécresse criticizing Macron for not implementing necessary reforms during his presidency. She highlights the need for reforms in civil service positions, unemployment insurance, and the RSA (a social benefit program). Zemmour sarcastically suggests taking away benefits from foreigners as a courageous reform, to which Pécresse argues that foreigners represent only a small percentage of benefit recipients.

The discussion returns to the retirement age, and the moderator asks Pécresse if she would implement a similar reform if she becomes president. Pécresse confirms her support for raising the retirement age to sixty-five but with exceptions for those who have had physically demanding careers. She also discusses her plans for unemployment insurance reform, reducing civil service positions, and increasing minimum pensions and widow's pensions.

Zemmour challenges Pécresse's commitment to fundamental reforms, questioning her ability to follow through on promises. He believes that previous promises of reducing public staff have been empty words. Pécresse acknowledges the need for sector-specific hiring and job eliminations but emphasizes the importance of careful assessment and planning.

The conversation briefly touches on decentralization and proposed budgetary savings, particularly related to non-European foreigners and fighting social fraud. Zemmour believes his program is fully funded through these measures, along with economic relief from charges and tax reductions. Pécresse dismisses these claims as empty promises and raises concerns about the funding of CSG (general social contribution) and hospitals in Zemmour's program.

Pécresse argues that unannounced reforms are less likely to be implemented, citing Nicolas Sarkozy's job cut announcements and her own successful implementation of job cuts in her region. Zemmour responds by comparing Pécresse to Macron and implying that their policies are similar. Pécresse emphasizes the importance of economic power and the need for reforms to address the country's financial challenges.

The conversation ends with Pécresse highlighting the increasing debt and the urgency of implementing reforms, while Zemmour maintains that reforms are unnecessary."""


Mafoi = tokenizer(leprompt, return_tensors="pt").to('cuda') # ICI RESTRICTION sur prompt
input_length = len(Mafoi['input_ids'][0])
print(input_length)

534


In [4]:
# Method and variables
# ==========================================================================================
print('Starting scores computation...')
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

STORAGE_FILE_NAME = 'scores'
PREPROCESS_SUMMARIES = True

# Script itself
# ==========================================================================================

# Find output file for CSV scores
mkdir('output')
mkdir('output/scores')
storage_file = open('output/scores/'+ 'scores' + '.csv', 'w', encoding='utf-8')
storage_file.write('path;rouge2;rougel;bertscore\n')

target_file_paths = []
references = []
predictions = []

# For each instruction
for instruction_n in range(n_instructions):

    # Read instruction and create prompt
    instruction = instructions[instruction_n]
    
    # For each sample in dataset
    for sample_n in range(n_samples):
        
        # Find target file
        target_file_path = 'intermediate/' + DATASET_NAME + '/' + str(instruction_n + 1) + '_' + str(sample_n + 1) + '.txt'
        if not os.path.isfile(target_file_path): # A MODIFIER : SI UN RESUME N'A PAS ETE GENERE
            print('-- Found no intermediate result file \'' + target_file_path + '\', skipped.')
            continue
        
        # Read sample and generate prompt -> Keep summary
        summary_file_path = 'input/' + DATASET_NAME + '/summaries/sample_' + str(sample_n + 1) + '.txt'
        summary_file = open(summary_file_path, 'r', encoding='utf-8')
        references.append(summary_file.read())
        summary_file.close()

        # Access generated summary
        target_file = open(target_file_path, 'r', encoding='utf-8')
        prediction = target_file.read()
        target_file.close()

        # Process answer
        if PREPROCESS_SUMMARIES:
            separator = "### Assistant:"
            prediction = prediction[prediction.index(separator) + len(separator):]
            if prediction[0] == " ": # Enlever l'espace devant
                prediction = prediction[1:]
            prediction = prediction[:-len("<|endoftext|>") - 2]
            #print(prediction)
            #print('---------------------------------------')
        
        # Add prediction
        target_file_paths.append(target_file_path)
        predictions.append(prediction)

# Calculate metrics
result_rouge = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
#result_bleu = bleu.compute(predictions=predictions, references=references)
result_bertscore = bertscore.compute(predictions=predictions, references=references, lang='fr', rescale_with_baseline=True, verbose=True)

# Write to csv
# Forget about BLEU...
# Format: PATH | ROUGE2 | ROUGEL | (BLEU |) BERTScore
for i in range(len(target_file_paths)):
    ligne = target_file_paths[i]
    ligne += ';' + str(result_rouge['rouge2'][i]) + ";" + str(result_rouge['rougeL'][i])
    #ligne += ";" + str(result_bleu['bleu'])
    ligne += ";" + str(result_bertscore['f1'][i])

    storage_file.write(ligne + '\n')

storage_file.close()
print('Done!')

Starting scores computation...
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 14839.71 seconds, 0.00 sentences/sec
Done!
